In [78]:
import pandas as pd
import numpy as np
import seaborn as sns
import math
from statsmodels.stats import diagnostic
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [79]:
data_folder='data/'

food_consumption = pd.read_csv(data_folder+'all/year_lsoa_grocery.csv')
print(len(food_consumption))
food_consumption.head(2)

4833


,area_id,weight,weight_perc2.5,weight_perc25,weight_perc50,weight_perc75,weight_perc97.5,weight_std,weight_ci95,volume,...,man_day,population,male,female,age_0_17,age_18_64,age_65+,avg_age,area_sq_km,people_per_sq_km
0,E01000001,308.119047,35.0,150.0,250.0,400.0,900.0,266.851723,2.359404,97.218600,...,16033,1296.0,685.0,611.0,179.0,766.0,351.0,48.293981,0.13,9985.0
1,E01000002,313.517874,40.0,150.0,250.0,400.0,900.0,256.227843,2.833812,88.262314,...,7829,1156.0,616.0,540.0,197.0,656.0,303.0,47.398789,0.23,5061.0


In [80]:
converters={'IMD Score': lambda s : np.float64(s.replace(',','.'))}
poverty_level=pd.read_csv(data_folder+'ID_2015_for_London.csv',sep=';', converters=converters)
poverty_level=poverty_level[['LSOA code (2011)','IMD Score']]\
                           .rename(columns ={'LSOA code (2011)':'area_id','IMD Score':'IMD'})
print(len(poverty_level))
poverty_level.head(2)

4835


,area_id,IMD
0,E01000001,6.1
1,E01000002,6.3


In [81]:
dataset=food_consumption.merge(poverty_level,on='area_id')
print(len(dataset))
dataset.head(2)

4833


,area_id,weight,weight_perc2.5,weight_perc25,weight_perc50,weight_perc75,weight_perc97.5,weight_std,weight_ci95,volume,...,population,male,female,age_0_17,age_18_64,age_65+,avg_age,area_sq_km,people_per_sq_km,IMD
0,E01000001,308.119047,35.0,150.0,250.0,400.0,900.0,266.851723,2.359404,97.218600,...,1296.0,685.0,611.0,179.0,766.0,351.0,48.293981,0.13,9985.0,6.1
1,E01000002,313.517874,40.0,150.0,250.0,400.0,900.0,256.227843,2.833812,88.262314,...,1156.0,616.0,540.0,197.0,656.0,303.0,47.398789,0.23,5061.0,6.3


In [77]:
mod = smf.ols(formula='IMD ~ fat + saturate +avg_age +people_per_sq_km+male+female +sugar + protein + carb + fibre + energy_tot + h_nutrients_calories',
              data=dataset)

# C(DEATH_EVENT,  Treatment(reference=0)) implies that we are considering the population that did not die!

res = mod.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                    IMD   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.517
Method:                 Least Squares   F-statistic:                     432.5
Date:                Sun, 06 Dec 2020   Prob (F-statistic):               0.00
Time:                        13:38:49   Log-Likelihood:                -17273.
No. Observations:                4833   AIC:                         3.457e+04
Df Residuals:                    4820   BIC:                         3.466e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              -39.2216 

In [ ]:
#id's (comlumn names) in the grocery dataset of the estimators to use
estimator_ids=['energy_tot','fat','saturate','sugar','protein','carb','fibre','h_nutrients_calories'] 

#names of the estimators, these are the names that will be used in the final graph
estimator_names=['energy','fat','saturate','sugar','protein','carb','fibre','entropy']
results=pd.DataFrame(data={'estimator_id':estimator_ids,'estimator_name':estimator_names})

In [ ]:
plt.style.use('ggplot')
fig, ax = plt.subplots(figsize=(7,5))
ax.set_title('Diabetes prevalence')

chart=sns.barplot(x="estimator_name", y="correlation", data=results,palette=color_palette)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90)

ax.axhline(0, color='black', lw=1)
plt.ylabel("R")
plt.xlabel(" ")
plt.show()